In [25]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
#print(check_output(["dir", "../sberbank housing prediction"],shell=True))   #print(check_output(["ls", "../input"]).decode("utf8"))  适用于 POSIX-conforming systems
#os.listdir()
print(os.listdir())
macro=pd.read_csv('macro.csv',parse_dates=['timestamp'])
sample_submission=pd.read_csv('sample_submission.csv')
test=pd.read_csv('test.csv',parse_dates=['timestamp'])
train=pd.read_csv('train.csv',parse_dates=['timestamp'])


['.git', '.ipynb_checkpoints', '3D-scatter-chart.ipynb', 'data_dictionary.txt', 'historical score.txt', 'macro.csv', 'RandomForest.ipynb', 'ridge regression test.ipynb', 'ridgeCV.ipynb', 'ridge_plot.png', 'run_at_here.bat', 'sample_submission.csv', 'test.csv', 'train.csv', '__MACOSX']


In [30]:
# 数据清洗以及特征选择
#随机森林是否接受不变成哑变量的回归
#对各种参数进行对比，并输出各种参数的三种score曲线
#添加更多有意义的变量，对非法值进行填充
#KFold

#相关性分析
#PCA


#保留没有空值的字段,以及非类别字段
# pd.options.display.max_seq_items = 1000   #控制是否完全打印
# temp=pd.DataFrame(train.isnull().sum(),columns=['numberofNA'])
# print (temp[temp['numberofNA']==0].index) #打印所有没有空值的字段

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestRegressor

test.loc[test['product_type'].isnull()==True,'product_type']='Investment'  #替换NA值

train_1=train[['id', 'timestamp', 'full_sq', 'area_m','product_type',
       'raion_popul', 'green_zone_part', 'indust_part', 'children_preschool',
       'preschool_education_centers_raion', 'children_school',
       'school_education_centers_raion',
       'school_education_centers_top_20_raion', 'healthcare_centers_raion',
       'university_top_20_raion', 'sport_objects_raion',
       'additional_education_raion',
       'culture_objects_top_25_raion', 'shopping_centers_raion',
       'office_raion',
       'full_all', 'male_f', 'female_f', 'young_all', 'young_male',
       'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all',
       'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female',
       '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male',
       '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all',
       '0_13_male', '0_13_female', 'ID_metro', 'metro_min_avto',
       'metro_km_avto', 'kindergarten_km', 'school_km', 'park_km',
       'green_zone_km', 'industrial_km', 'water_treatment_km', 'cemetery_km',
       'incineration_km', 'railroad_station_avto_km',
       'railroad_station_avto_min', 'ID_railroad_station_avto',
       'public_transport_station_km', 'public_transport_station_min_walk',
       'water_km', 'mkad_km', 'ttk_km', 'sadovoe_km',
       'bulvar_ring_km', 'kremlin_km', 'big_road1_km', 'ID_big_road1', 'big_road2_km', 'ID_big_road2', 'railroad_km', 'zd_vokzaly_avto_km', 'ID_railroad_terminal',
       'bus_terminal_avto_km', 'ID_bus_terminal', 'oil_chemistry_km',
       'nuclear_reactor_km', 'radiation_km', 'power_transmission_line_km',
       'thermal_power_plant_km', 'ts_km', 'big_market_km', 'market_shop_km',
       'fitness_km', 'swim_pool_km', 'ice_rink_km', 'stadium_km',
       'basketball_km', 'hospice_morgue_km', 'detention_facility_km',
       'public_healthcare_km', 'university_km', 'workplaces_km',
       'shopping_centers_km', 'office_km', 'additional_education_km',
       'preschool_km', 'big_church_km', 'church_synagogue_km', 'mosque_km',
       'theater_km', 'museum_km', 'exhibition_km', 'catering_km',
       'green_part_500', 'prom_part_500', 'office_count_500', 'office_sqm_500',
       'trc_count_500', 'trc_sqm_500', 'cafe_count_500',
       'cafe_count_500_na_price', 'cafe_count_500_price_500',
       'cafe_count_500_price_1000', 'cafe_count_500_price_1500',
       'cafe_count_500_price_2500', 'cafe_count_500_price_4000',
       'cafe_count_500_price_high', 'big_church_count_500', 'church_count_500',
       'mosque_count_500', 'leisure_count_500', 'sport_count_500',
       'market_count_500', 'green_part_1000', 'prom_part_1000',
       'office_count_1000', 'office_sqm_1000', 'trc_count_1000',
       'trc_sqm_1000', 'cafe_count_1000', 'cafe_count_1000_na_price',
       'cafe_count_1000_price_500', 'cafe_count_1000_price_1000',
       'cafe_count_1000_price_1500', 'cafe_count_1000_price_2500',
       'cafe_count_1000_price_4000', 'cafe_count_1000_price_high',
       'big_church_count_1000', 'church_count_1000', 'mosque_count_1000',
       'leisure_count_1000', 'sport_count_1000', 'market_count_1000',
       'green_part_1500', 'prom_part_1500', 'office_count_1500',
       'office_sqm_1500', 'trc_count_1500', 'trc_sqm_1500', 'cafe_count_1500',
       'cafe_count_1500_na_price', 'cafe_count_1500_price_500',
       'cafe_count_1500_price_1000', 'cafe_count_1500_price_1500',
       'cafe_count_1500_price_2500', 'cafe_count_1500_price_4000',
       'cafe_count_1500_price_high', 'big_church_count_1500',
       'church_count_1500', 'mosque_count_1500', 'leisure_count_1500',
       'sport_count_1500', 'market_count_1500',
       'prom_part_2000', 'office_count_2000', 'office_sqm_2000',
       'trc_count_2000', 'trc_sqm_2000', 'cafe_count_2000',
       'cafe_count_2000_na_price', 'cafe_count_2000_price_500',
       'cafe_count_2000_price_1000', 'cafe_count_2000_price_1500',
       'cafe_count_2000_price_2500', 'cafe_count_2000_price_4000',
       'cafe_count_2000_price_high', 'big_church_count_2000',
       'church_count_2000', 'mosque_count_2000', 'leisure_count_2000',
       'sport_count_2000', 'market_count_2000', 'green_part_3000',
       'prom_part_3000', 'office_count_3000', 'office_sqm_3000',
       'trc_count_3000', 'trc_sqm_3000', 'cafe_count_3000',
       'cafe_count_3000_na_price', 'cafe_count_3000_price_500',
       'cafe_count_3000_price_1000', 'cafe_count_3000_price_1500',
       'cafe_count_3000_price_2500', 'cafe_count_3000_price_4000',
       'cafe_count_3000_price_high', 'big_church_count_3000',
       'church_count_3000', 'mosque_count_3000', 'leisure_count_3000',
       'sport_count_3000', 'market_count_3000', 'green_part_5000',
       'office_count_5000', 'office_sqm_5000', 'trc_count_5000',
       'trc_sqm_5000', 'cafe_count_5000', 'cafe_count_5000_na_price',
       'cafe_count_5000_price_500', 'cafe_count_5000_price_1000',
       'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500',
       'cafe_count_5000_price_4000', 'cafe_count_5000_price_high',
       'big_church_count_5000', 'church_count_5000', 'mosque_count_5000',
       'leisure_count_5000', 'sport_count_5000', 'market_count_5000','price_doc']]
test_1=test[['id', 'timestamp', 'full_sq', 'area_m','product_type',
       'raion_popul', 'green_zone_part', 'indust_part', 'children_preschool',
       'preschool_education_centers_raion', 'children_school',
       'school_education_centers_raion',
       'school_education_centers_top_20_raion', 'healthcare_centers_raion',
       'university_top_20_raion', 'sport_objects_raion',
       'additional_education_raion',
       'culture_objects_top_25_raion', 'shopping_centers_raion',
       'office_raion',
       'full_all', 'male_f', 'female_f', 'young_all', 'young_male',
       'young_female', 'work_all', 'work_male', 'work_female', 'ekder_all',
       'ekder_male', 'ekder_female', '0_6_all', '0_6_male', '0_6_female',
       '7_14_all', '7_14_male', '7_14_female', '0_17_all', '0_17_male',
       '0_17_female', '16_29_all', '16_29_male', '16_29_female', '0_13_all',
       '0_13_male', '0_13_female', 'ID_metro', 'metro_min_avto',
       'metro_km_avto', 'kindergarten_km', 'school_km', 'park_km',
       'green_zone_km', 'industrial_km', 'water_treatment_km', 'cemetery_km',
       'incineration_km', 'railroad_station_avto_km',
       'railroad_station_avto_min', 'ID_railroad_station_avto',
       'public_transport_station_km', 'public_transport_station_min_walk',
       'water_km', 'mkad_km', 'ttk_km', 'sadovoe_km',
       'bulvar_ring_km', 'kremlin_km', 'big_road1_km', 'ID_big_road1', 'big_road2_km', 'ID_big_road2', 'railroad_km', 'zd_vokzaly_avto_km', 'ID_railroad_terminal',
       'bus_terminal_avto_km', 'ID_bus_terminal', 'oil_chemistry_km',
       'nuclear_reactor_km', 'radiation_km', 'power_transmission_line_km',
       'thermal_power_plant_km', 'ts_km', 'big_market_km', 'market_shop_km',
       'fitness_km', 'swim_pool_km', 'ice_rink_km', 'stadium_km',
       'basketball_km', 'hospice_morgue_km', 'detention_facility_km',
       'public_healthcare_km', 'university_km', 'workplaces_km',
       'shopping_centers_km', 'office_km', 'additional_education_km',
       'preschool_km', 'big_church_km', 'church_synagogue_km', 'mosque_km',
       'theater_km', 'museum_km', 'exhibition_km', 'catering_km',
       'green_part_500', 'prom_part_500', 'office_count_500', 'office_sqm_500',
       'trc_count_500', 'trc_sqm_500', 'cafe_count_500',
       'cafe_count_500_na_price', 'cafe_count_500_price_500',
       'cafe_count_500_price_1000', 'cafe_count_500_price_1500',
       'cafe_count_500_price_2500', 'cafe_count_500_price_4000',
       'cafe_count_500_price_high', 'big_church_count_500', 'church_count_500',
       'mosque_count_500', 'leisure_count_500', 'sport_count_500',
       'market_count_500', 'green_part_1000', 'prom_part_1000',
       'office_count_1000', 'office_sqm_1000', 'trc_count_1000',
       'trc_sqm_1000', 'cafe_count_1000', 'cafe_count_1000_na_price',
       'cafe_count_1000_price_500', 'cafe_count_1000_price_1000',
       'cafe_count_1000_price_1500', 'cafe_count_1000_price_2500',
       'cafe_count_1000_price_4000', 'cafe_count_1000_price_high',
       'big_church_count_1000', 'church_count_1000', 'mosque_count_1000',
       'leisure_count_1000', 'sport_count_1000', 'market_count_1000',
       'green_part_1500', 'prom_part_1500', 'office_count_1500',
       'office_sqm_1500', 'trc_count_1500', 'trc_sqm_1500', 'cafe_count_1500',
       'cafe_count_1500_na_price', 'cafe_count_1500_price_500',
       'cafe_count_1500_price_1000', 'cafe_count_1500_price_1500',
       'cafe_count_1500_price_2500', 'cafe_count_1500_price_4000',
       'cafe_count_1500_price_high', 'big_church_count_1500',
       'church_count_1500', 'mosque_count_1500', 'leisure_count_1500',
       'sport_count_1500', 'market_count_1500',
       'prom_part_2000', 'office_count_2000', 'office_sqm_2000',
       'trc_count_2000', 'trc_sqm_2000', 'cafe_count_2000',
       'cafe_count_2000_na_price', 'cafe_count_2000_price_500',
       'cafe_count_2000_price_1000', 'cafe_count_2000_price_1500',
       'cafe_count_2000_price_2500', 'cafe_count_2000_price_4000',
       'cafe_count_2000_price_high', 'big_church_count_2000',
       'church_count_2000', 'mosque_count_2000', 'leisure_count_2000',
       'sport_count_2000', 'market_count_2000', 'green_part_3000',
       'prom_part_3000', 'office_count_3000', 'office_sqm_3000',
       'trc_count_3000', 'trc_sqm_3000', 'cafe_count_3000',
       'cafe_count_3000_na_price', 'cafe_count_3000_price_500',
       'cafe_count_3000_price_1000', 'cafe_count_3000_price_1500',
       'cafe_count_3000_price_2500', 'cafe_count_3000_price_4000',
       'cafe_count_3000_price_high', 'big_church_count_3000',
       'church_count_3000', 'mosque_count_3000', 'leisure_count_3000',
       'sport_count_3000', 'market_count_3000', 'green_part_5000',
       'office_count_5000', 'office_sqm_5000', 'trc_count_5000',
       'trc_sqm_5000', 'cafe_count_5000', 'cafe_count_5000_na_price',
       'cafe_count_5000_price_500', 'cafe_count_5000_price_1000',
       'cafe_count_5000_price_1500', 'cafe_count_5000_price_2500',
       'cafe_count_5000_price_4000', 'cafe_count_5000_price_high',
       'big_church_count_5000', 'church_count_5000', 'mosque_count_5000',
       'leisure_count_5000', 'sport_count_5000', 'market_count_5000']]


macro_1=macro[['timestamp','oil_urals','gdp_annual','gdp_annual_growth','average_provision_of_build_contract',
'deposits_value','mortgage_value','mortgage_rate','fixed_basket']]


#将类别变量变换为哑变量
dummies = pd.get_dummies(train_1['product_type'], prefix='product_type')
train_1=train_1.join(dummies).drop('product_type',axis=1)

dummies = pd.get_dummies(test_1['product_type'], prefix='product_type')
test_1=test_1.join(dummies).drop('product_type',axis=1)


In [31]:
train_withmacro=pd.merge(train_1,macro_1,on='timestamp')
test_withmacro=pd.merge(test_1,macro_1,on='timestamp')
samples_y=train_withmacro['price_doc']
samples_x=train_withmacro.drop(['price_doc'],axis=1)



#标准化

samples_x=samples_x.drop(['id','timestamp'], axis=1)
test_x=test_withmacro.drop(['id','timestamp'], axis=1)

normalize=Normalizer()   
scaler=StandardScaler()



samples_x=scaler.fit_transform(samples_x)
test_x=scaler.transform(test_x)
# samples_x=normalize.fit_transform(samples_x)
# test_x=normalize.transform(test_x)
# samples_x=rfr.fit_transform(samples_x,None)
# test_x=rfr.transform(test_x)


In [32]:
from sklearn.cross_validation import train_test_split
temp_x,verify_x,temp_y,verify_y=train_test_split(samples_x,samples_y,test_size=0.25,random_state=42)
train_x,validation_x,train_y,validation_y=train_test_split(temp_x,temp_y,test_size=0.5,random_state=42)


In [33]:
rfr = RandomForestRegressor(n_estimators=20,oob_score=True)
rfr.fit(train_x,train_y)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=20, n_jobs=1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [34]:
#cross validation evaluation
print ('the score for train set itself:',rfr.score(train_x,train_y))
print ('the score for validation set:',rfr.score(validation_x,validation_y))
print ('the score for verify set:',rfr.score(verify_x,verify_y))

the score for train set itself: 0.941818581662
the score for validation set: 0.633961064075
the score for verify set: 0.635985010672


In [23]:
# output the prediction
rfr.predict(test_x)


array([  5144105.05 ,   8725710.325,   5461000.   , ...,   4727374.05 ,
         5523000.   ,  10563929.35 ])

In [24]:
rfr.apply(train_x)
rfr.get_params(deep=True)
rfr.oob_score_
import sklearn
print('Your sklearn version is {}'.format(sklearn.__version__))

Your sklearn version is 0.17.1


In [22]:
test_x.shape

(7662, 232)